In [1]:
import re
import pandas as pd
import numpy as np
import simplemma
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
stop_words = get_stop_words('uk')

In [3]:
#попередня обробка тексту
def ClearText(text):
    #переведення до нижнього регістру всіх слів
    cleartext = text.lower()
    #print(cleartext)
    #прибирання пустих рядків та розрив рядків
    cleartext = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', cleartext) 
    #залишаємо лише слова, прибираємо пунктуацію та числа
    cleartext = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', cleartext) #deleting symbols  
    #cleartext = cleartext.translate(remove_digits)
    cleartext = cleartext.replace("\\", "")
    cleartext = cleartext.rstrip()
    #прибираємо зайві пробіи
    cleartext = re.sub(" +", " ", cleartext)
    #ділимо речення на список слів, розбиваємо по пробілам
    cleartext = re.split(" ", cleartext)
    #прибираємо стопслова
    cleartext = [word for word in cleartext if word not in stop_words]
    #прибираємо слова, довжина який менше 3 букв
    cleartext = [word for word in cleartext if len(word) > 3]
    #лематизація слів
    cleartext = [simplemma.lemmatize(word, lang='uk') for word in cleartext]
    return ' '.join(cleartext)

In [4]:
print(stop_words)

['a', 'б', 'в', 'г', 'е', 'ж', 'з', 'м', 'т', 'у', 'я', 'є', 'і', 'аж', 'ви', 'де', 'до', 'за', 'зі', 'ми', 'на', 'не', 'ну', 'нх', 'ні', 'по', 'та', 'ти', 'то', 'ту', 'ті', 'це', 'цю', 'ця', 'ці', 'чи', 'ще', 'що', 'як', 'їй', 'їм', 'їх', 'її', 'або', 'але', 'ало', 'без', 'був', 'вам', 'вас', 'ваш', 'вже', 'все', 'всю', 'вся', 'від', 'він', 'два', 'дві', 'для', 'ким', 'мож', 'моя', 'моє', 'мої', 'міг', 'між', 'мій', 'над', 'нам', 'нас', 'наш', 'нею', 'неї', 'них', 'ніж', 'ній', 'ось', 'при', 'про', 'під', 'пір', 'раз', 'рік', 'сам', 'сих', 'сім', 'так', 'там', 'теж', 'тим', 'тих', 'той', 'тою', 'три', 'тут', 'хоч', 'хто', 'цей', 'цим', 'цих', 'час', 'щоб', 'яка', 'які', 'адже', 'буде', 'буду', 'будь', 'була', 'були', 'було', 'бути', 'вами', 'ваша', 'ваше', 'ваші', 'весь', 'вниз', 'вона', 'вони', 'воно', 'всею', 'всім', 'всіх', 'втім', 'геть', 'далі', 'двох', 'день', 'дуже', 'зате', 'його', 'йому', 'каже', 'кого', 'коли', 'кому', 'крім', 'куди', 'лише', 'люди', 'мало', 'мати', 'мене', 

In [5]:
df_train = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_train.xlsx')
df_test = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_test.xlsx')
df_train.head()

,Comment,Category
0,Що вам найбільше сподобалося в перевтіленні Лю...,Not_Toxic
1,"яка гарна родина, щастя вам))",Not_Toxic
2,Дякую команді за такий чудовий проєкт. На жаль...,Not_Toxic
3,"Щодо танго не згодна. Виглядає так, наче Люда ...",Not_Toxic
4,"Людмила дуже гарна жінка, їй дуже пасує жіночн...",Not_Toxic


In [41]:
df = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_all.xlsx')
df = df.sample(frac=1) 
df

,Comment,Category
934,"Юначе, у Вас така милозву чна украинська мова,...",Not_Toxic
580,"Вибачте, але івасі - це оселедець, оселедець т...",Not_Toxic
46,"На обіцяли в ЄС, давайте спочатку в ЄС вступим...",Not_Toxic
162,Дякую!,Not_Toxic
991,А може не на часi розповiдати ?,Not_Toxic
...,...,...
97,"Людмила чудова людина, має прикрасну сім'ю, чу...",Not_Toxic
428,"Якщо приїхати до Львова росіянину, тебе не вб'...",Toxic
871,А чого позаду вас Маріуполь російською? Хлопці...,Toxic
698,"Дякую за чудову мову, приємно слухать!",Not_Toxic


In [42]:
df_train = df.iloc [:round(df.shape[0]*0.75)]
df_test = df.iloc [round(df.shape[0]*0.75):]

In [43]:
df_train['ClearText'] = df_train.apply(lambda x: ClearText(x['Comment']), axis=1)
df_test['ClearText'] = df_test.apply(lambda x: ClearText(x['Comment']), axis=1)

C:\Users\helen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\helen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
df_test

,Comment,Category,ClearText
196,"Класний випуск, адже нарешті показали іншу сто...",Not_Toxic,класний випуск показати інший сторона гламуру ...
1143,"темно у неї під очима через часті секси, або ч...",Toxic,темно око частий секси частий мастурбації
870,Ваша українська то оргазм для вух,Not_Toxic,український оргазм
923,"Інформацію відкривали вірну, але не все вірно ...",Not_Toxic,інформація відкривати вірний вірно трактувати ...
718,Ви прекрасні.Здоров'я всій вашій родині.,Not_Toxic,прекрасний здоров'я весь ваш родина
...,...,...,...
1075,Дякую Любочко за цікаву гостю і питання по Сум...,Not_Toxic,любочко цікавий гостя питання сумщині людочко ...
121,Фотосесія відкрила таку неперевершену жіночніс...,Not_Toxic,фотосесія відкрити така неперевершену жіночніс...
592,Мова - код нації! Дякую за таку роботу! Супер....,Not_Toxic,мова нація така робот супер вчити раз
485,Останній раунд не дуже подобається,Not_Toxic,останній раунд подобатися


In [44]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df_train['ClearText'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# MultinomialNB

In [63]:
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(df_train['ClearText'], df_train['Category'])

In [64]:
predicted = text_clf.predict(df_test['ClearText'])
np.mean(predicted == df_test['Category'])

0.6804123711340206

# SVM

In [39]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3,  random_state=42)),
])
train_svm = text_clf_svm.fit(df_train['ClearText'], df_train['Category'])

In [40]:
predicted_svm = text_clf_svm.predict(df_test['ClearText'])
np.mean(predicted_svm == df_test['Category'])

0.7628865979381443

In [67]:
predicted_svm

array(['Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', '

# LogisticRegression

In [45]:
text_clf_log_reg = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)),
])
train_log_reg = text_clf_log_reg.fit(df_train['ClearText'], df_train['Category'])

[LibLinear]

In [46]:
predicted_log_reg = text_clf_log_reg.predict(df_test['ClearText'])
np.mean(predicted_log_reg == df_test['Category'])

0.7697594501718213

In [73]:
text_clf_log_reg1 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=20, penalty='l2',  max_iter=1000)),
])
train_log_reg1 = text_clf_log_reg1.fit(df_train['ClearText'], df_train['Category'])
predicted_log_reg1 = text_clf_log_reg1.predict(df_test['ClearText'])
np.mean(predicted_log_reg1 == df_test['Category'])

[LibLinear]

0.7766323024054983

In [47]:
predicted_log_reg

array(['Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_T

In [48]:
predicted_log_reg_df = pd.DataFrame(predicted_log_reg, columns=['predictions'])
df_test.index = range(0, df_test.shape[0])
result = pd.concat([predicted_log_reg_df, df_test], axis=1)
result.to_csv('C:/Users/helen/OneDrive/Рабочий стол/пары/prediction.csv')
result

,predictions,Comment,Category,ClearText
0,Not_Toxic,Його слухати те ж що наш зомбоящик..що він мож...,Toxic,слухати зомбоящик знати незрозуміло
1,Not_Toxic,А КИЇВ СКЛЯНА ГОСТИНИЦА ПОСТРАЖДАЛА І ЦЕ БУЛО ...,Not_Toxic,кий скляний гостиница постраждалий біля вокзал
2,Not_Toxic,УКРАЇНА ПЕРЕМОЖЕ СЛАВА УКРАЇНІ СЛАВА ЗСУ,Not_Toxic,Україна перемогти слава Україна слава
3,Not_Toxic,Найкраща підбірка пісень. Скрябін,Not_Toxic,найкращий підбірка пісня скрябін
4,Not_Toxic,Красива жінка Будьте щасливі Самій захотілось ...,Not_Toxic,красивий жінка бути щасливий захотітися така ф...
...,...,...,...,...
286,Not_Toxic,"Людмила чудова людина, має прикрасну сім'ю, чу...",Not_Toxic,Людмила чудовий прикрасну сім'я чудовий чолові...
287,Not_Toxic,"Якщо приїхати до Львова росіянину, тебе не вб'...",Toxic,приїхати Львів росіянину вбити
288,Toxic,А чого позаду вас Маріуполь російською? Хлопці...,Toxic,позаду маріуполь російською хлопець
289,Not_Toxic,"Дякую за чудову мову, приємно слухать!",Not_Toxic,чудовий мова приємно слухати


# Metrics

In [49]:
pred_labels = np.asarray(list(result['predictions']))
true_labels = np.asarray(list(result['Category']))

In [50]:
# True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
TP = np.sum(np.logical_and(pred_labels == 'Toxic', true_labels == 'Toxic'))
 
# True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
TN = np.sum(np.logical_and(pred_labels == 'Not_Toxic', true_labels == 'Not_Toxic'))
 
# False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
FP = np.sum(np.logical_and(pred_labels == 'Toxic', true_labels == 'Not_Toxic'))
 
# False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
FN = np.sum(np.logical_and(pred_labels == 'Not_Toxic', true_labels == 'Toxic'))

In [51]:
print('TP = {}, TN = {}, FP = {}, FN = {}'.format(TP, TN, FP, FN))


TP = 39, TN = 185, FP = 13, FN = 54


In [26]:
list(true_labels).count('Toxic')

87

In [27]:
list(true_labels).count('Not_Toxic')

204

In [52]:
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1_Score = 2 * (Precision * Recall) / (Precision + Recall)
F1_Score

0.5379310344827586